In [1]:
#@title Mount Drive
import os
if os.path.ismount('/content/drive'):
    print("Google Drive is already mounted.")
else:
    from google.colab import drive
    drive.mount('/content/drive')

sample_data


In [13]:
from pathlib import Path
import shlex

In [ ]:
NOTEBOOK_DIR = Path("/content/drive/MyDrive/Colab Notebooks/vad_clip")
NOTEBOOK_NAME = "vad_clip_notebook.ipynb"
NOTEBOOK_PATH = NOTEBOOK_DIR / NOTEBOOK_NAME

with open(NOTEBOOK_DIR / "vad_clip_gh_token.txt") as f:
  gh_token = f.read().strip()

REPO = "vad_clip_colab_notebook"
GITHUB_URL = f"https://{gh_token}@github.com/hammad-ali1/{REPO}.git"

In [40]:
!git clone {GITHUB_URL}
!cp "{NOTEBOOK_PATH}" {REPO}/{NOTEBOOK_NAME}

fatal: destination path 'vad_clip_colab_notebook' already exists and is not an empty directory.


In [38]:
!ls "{NOTEBOOK_PATH}"

'/content/drive/MyDrive/Colab Notebooks/vad_clip/vad_clip_notebook.ipynb'
